### This notebook creates aggregated PAS survey used for the main trust and confidence predictiosn based on the crime counts

In [2]:
# imports 

import pandas as pd
import numpy as np

import seaborn as sns

import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from tqdm import tqdm

# Files for Trust

# PAS data

In [3]:
# download the data

data_15_17 = pd.read_csv(".\data\PAS_ward_level_FY_15_17.csv")
data_17_18 = pd.read_csv(".\data\PAS_ward_level_FY_17_18.csv")
data_18_19 = pd.read_csv(".\data\PAS_ward_level_FY_18_19.csv")
data_19_20 = pd.read_csv(".\data\PAS_ward_level_FY_19_20.csv")
data_20_21 = pd.read_csv(".\data\PAS_ward_level_FY_20_21.csv")


C:\Users\Marceli Morawski\AppData\Local\Temp\ipykernel_55252\2865077880.py:3: DtypeWarning: Columns (13,14,15,16,17,18,19,20,21,22,35,177,410,411,412,415,416,417,418,419,420,421,422,423) have mixed types. Specify dtype option on import or set low_memory=False.
  data_15_17 = pd.read_csv(".\data\PAS_ward_level_FY_15_17.csv")
C:\Users\Marceli Morawski\AppData\Local\Temp\ipykernel_55252\2865077880.py:4: DtypeWarning: Columns (30,31,32,98,99,100,101,102,103,104,105,125,126,127,199,200) have mixed types. Specify dtype option on import or set low_memory=False.
  data_17_18 = pd.read_csv(".\data\PAS_ward_level_FY_17_18.csv")
C:\Users\Marceli Morawski\AppData\Local\Temp\ipykernel_55252\2865077880.py:5: DtypeWarning: Columns (25,26,27,28,108,109,110,112,113,137,202,203,381) have mixed types. Specify dtype option on import or set low_memory=False.
  data_18_19 = pd.read_csv(".\data\PAS_ward_level_FY_18_19.csv")
C:\Users\Marceli Morawski\AppData\Local\Temp\ipykernel_55252\2865077880.py:6: DtypeWa

In [4]:
# functions for transformation

### function for encoding ordinal and ordinal "like" data

def ordinal_encoder(data: pd.DataFrame, category: list[str], column: str, missing: int):
    """Function encodes ordinal varaibles"""
    encoder = OrdinalEncoder(categories=category, handle_unknown='use_encoded_value', unknown_value=missing)
    encoder.fit(data[[column]])
    data[column] = encoder.transform(data[[column]])


def month_transformation(month):
    month_info = {
        'Jan': 1,
        'Feb': 2, 
        'Mar': 3,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'Nov': 11,
        'Dec': 12
    }
    return  month_info[month]

def data_creation(row, col):
    """creates date from the available columns"""
    month = row[col].split()[1][1:]
    year = row[col].split()[2][:-1]
    date_str = f'01/{month_transformation(month)}/{year}' # day of an interview unvailable -> always equal to 01
    date = pd.to_datetime(date_str,  format='%d/%m/%Y')
    return date


### computes proporrion for PAS trust - uses scale
def ProportionComputation1(inner: dict):

    scale = {0.0:0, 1.0:0.25, 2.0:0.5, 3.0:0.75, 4.0:1}
    total = 0
    number = 0
    for label in sorted(inner.keys())[1:]:
        number += inner[label]
        total += inner[label] * scale[label]

    return np.round(total/number, 2)

### computes proporrion for PAS trust - computes proportion of positive responses
def ProportionComputation2(inner: dict)
    number = 0
    total = inner[3] + inner[4]
    for label in sorted(inner.keys())[1:]:
        number += inner[label] 

    return np.round(total/number, 2)
 
# creates data frame using ProportionComputation1 used with PAS
def PASPropportionDataFrame1(data: dict):
    columnDate = []
    columnProportion = []
    for key, value in data.items():
        columnDate.append(key)
        columnProportion.append(ProportionComputation1(value))
    layout = {"Date":columnDate, "Proportion":columnProportion}
    return pd.DataFrame.from_dict(layout)

# creates data frame using ProportionComputation2 used with PAS
def PASPropportionDataFrame2(data: dict):
    columnDate = []
    columnProportion = []
    for key, value in data.items():
        columnDate.append(key)
        columnProportion.append(ProportionComputation2(value))
    layout = {"Date":columnDate, "Proportion":columnProportion}
    return pd.DataFrame.from_dict(layout)



In [5]:
# applies datetime

data_15_17["datetime"] = data_15_17.apply(data_creation, axis=1, args=("MONTH", ))
data_17_18["datetime"] = data_17_18.apply(data_creation, axis=1, args=("MONTH", ))
data_18_19["datetime"] = data_18_19.apply(data_creation, axis=1, args=("MONTH", ))
data_19_20["datetime"] = data_19_20.apply(data_creation, axis=1, args=("MONTH", ))
data_20_21["datetime"] = data_20_21.apply(data_creation, axis=1, args=("MONTH", ))

In [54]:

selectecColumns = ["datetime", "NQ135BD"]

NQ135BD = [["Strongly disagree", "Tend to disagree", "Neither agree nor disagree", "Tend to agree", "Strongly agree"]]

data_15_17_trust = data_15_17[selectecColumns]
data_17_18_trust = data_17_18[selectecColumns]
data_18_19_trust = data_18_19[selectecColumns]
data_19_20_trust = data_19_20[selectecColumns]
data_20_21_trust = data_20_21[selectecColumns]

frames = [data_15_17_trust, data_17_18_trust, data_18_19_trust, data_19_20_trust, data_20_21_trust] 

trustData = pd.concat(frames)

# quick conversion to numbers for trust
ordinal_encoder(trustData, NQ135BD, "NQ135BD", -1)

trustData['datetime'] = trustData['datetime'].dt.to_period('M')

trustData.head()


,datetime,NQ135BD
0,2015-04,3.0
1,2015-04,4.0
2,2015-04,4.0
3,2015-04,4.0
4,2015-05,4.0


In [55]:
# we get count sfor each repsonse type per month

pd.options.display.max_rows = 50

grouped = trustData.groupby(["datetime"])["NQ135BD"].value_counts()

# getting the counts for the labels
result_dict = {}
for (date, value), count in grouped.items():
    if date not in result_dict:
        result_dict[date] = {}
    if value == -1: 
        pass
    else:
        result_dict[date][value] = count

print(result_dict)

{Period('2015-04', 'M'): {3.0: 511, 4.0: 353, 2.0: 99, 1.0: 46, 0.0: 20}, Period('2015-05', 'M'): {3.0: 523, 4.0: 388, 2.0: 96, 1.0: 37, 0.0: 14}, Period('2015-06', 'M'): {3.0: 577, 4.0: 377, 2.0: 95, 1.0: 48, 0.0: 15}, Period('2015-07', 'M'): {3.0: 527, 4.0: 385, 2.0: 97, 1.0: 30, 0.0: 17}, Period('2015-08', 'M'): {3.0: 523, 4.0: 361, 2.0: 85, 1.0: 31, 0.0: 26}, Period('2015-09', 'M'): {3.0: 517, 4.0: 429, 2.0: 108, 1.0: 31, 0.0: 26}, Period('2015-10', 'M'): {3.0: 455, 4.0: 400, 2.0: 92, 1.0: 46, 0.0: 31}, Period('2015-11', 'M'): {3.0: 523, 4.0: 404, 2.0: 81, 1.0: 39, 0.0: 19}, Period('2015-12', 'M'): {3.0: 568, 4.0: 414, 2.0: 84, 1.0: 23, 0.0: 12}, Period('2016-01', 'M'): {3.0: 589, 4.0: 348, 2.0: 71, 1.0: 36, 0.0: 11}, Period('2016-02', 'M'): {3.0: 1115, 4.0: 819, 2.0: 144, 1.0: 52, 0.0: 28}, Period('2016-03', 'M'): {3.0: 1084, 4.0: 778, 2.0: 171, 1.0: 46, 0.0: 34}, Period('2016-04', 'M'): {3.0: 545, 4.0: 415, 2.0: 72, 1.0: 22, 0.0: 12}, Period('2016-05', 'M'): {3.0: 525, 4.0: 414, 

In [65]:
# checks for availbale months
dataPasModel = PASPropportionDataFrame2(result_dict)
dataPasModel["Date"].unique()

<PeriodArray>
['2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09', '2015-10',
 '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-04', '2016-05',
 '2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12',
 '2017-01', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09',
 '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03', '2018-04',
 '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11',
 '2018-12', '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
 '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01',
 '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08',
 '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03']
Length: 70, dtype: period[M]

### PAS data is missing 2 months of the data from 2017-02 and 2017-03

# Crime data - aggreagation

In [3]:
dataCrime =  pd.read_csv(".\data\crimes.csv")

C:\Users\Marceli Morawski\AppData\Local\Temp\ipykernel_29668\2678658284.py:1: DtypeWarning: Columns (0,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  dataCrime =  pd.read_csv(".\data\crimes.csv")


In [4]:
dataCrime.head()

,Crime ID,Month,Quartile,Year,Crime type,Last outcome category,boroughs
0,NaN,4,Q2,2011,Anti-social behaviour,NaN,Lewisham
1,NaN,4,Q2,2011,Anti-social behaviour,NaN,Lewisham
2,NaN,4,Q2,2011,Anti-social behaviour,NaN,Lewisham
3,NaN,4,Q2,2011,Anti-social behaviour,NaN,Lewisham
4,NaN,4,Q2,2011,Anti-social behaviour,NaN,Lewisham


In [162]:
dataCrime[['Crime type']].value_counts()

Crime type                  
Anti-social behaviour           3701300
Violence and sexual offences    2293640
Other theft                     1481995
Vehicle crime                   1297916
Burglary                         958349
Criminal damage and arson        703094
Shoplifting                      513798
Public order                     509156
Drugs                            498353
Other crime                      480099
Theft from the person            468702
Robbery                          382677
Violent crime                    336990
Bicycle theft                    202332
Possession of weapons             53624
Public disorder and weapons       49907
Name: count, dtype: int64

In [36]:
# creation of data takes a lot  of time for crime data set, thus in next cell this data is being saved
# to avoid additional waiting time, ucomment this cell and downoald the transformed data
tqdm.pandas()

def CreateDate(row):
    date_str = str(str(row['Year']) + "-" + str(row['Month']))
    return pd.to_datetime(date_str,  format='%Y-%m')

dataCrime["date"] = dataCrime.progress_apply(CreateDate, axis=1)



100%|██████████| 21570944/21570944 [30:42<00:00, 11707.33it/s] 


In [37]:
# uncommetn this line when transformed data is not available

# dataCrime.to_csv('Crime.csv', index=False)

In [44]:
# Uncomment if data file if transformed dta is availbale
# dataCrime = pd.read_csv(".\data\Crime.csv")

dataCrime['date'] = pd.to_datetime(dataCrime['date'])

dataCrime['date'] = dataCrime['date'].dt.to_period('M')

C:\Users\Marceli Morawski\AppData\Local\Temp\ipykernel_55252\794255741.py:1: DtypeWarning: Columns (0,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  dataCrime = pd.read_csv(".\data\Crime.csv")


In [45]:
grouped_crime = dataCrime.groupby("date")["Crime type"].value_counts()

# getting the counts for the labels
result_dict_crime = {}
for (date, value), count in grouped_crime.items():
    if date not in result_dict_crime:
        result_dict_crime[date] = {}
    result_dict_crime[date][value] = count

print(result_dict_crime)

{Period('2010-12', 'M'): {'Anti-social behaviour': 34459, 'Other crime': 28104, 'Violent crime': 11784, 'Burglary': 7897, 'Vehicle crime': 7388, 'Robbery': 2791}, Period('2011-01', 'M'): {'Anti-social behaviour': 33542, 'Other crime': 32553, 'Violent crime': 12531, 'Burglary': 9020, 'Vehicle crime': 8156, 'Robbery': 3253}, Period('2011-02', 'M'): {'Anti-social behaviour': 32073, 'Other crime': 31806, 'Violent crime': 11852, 'Burglary': 7774, 'Vehicle crime': 7532, 'Robbery': 3113}, Period('2011-03', 'M'): {'Other crime': 35999, 'Anti-social behaviour': 35794, 'Violent crime': 13015, 'Vehicle crime': 8450, 'Burglary': 8313, 'Robbery': 3253}, Period('2011-04', 'M'): {'Anti-social behaviour': 40922, 'Other crime': 33758, 'Violent crime': 13305, 'Vehicle crime': 8192, 'Burglary': 7596, 'Robbery': 3223}, Period('2011-05', 'M'): {'Anti-social behaviour': 40510, 'Other crime': 37343, 'Violent crime': 13531, 'Vehicle crime': 9182, 'Burglary': 8210, 'Robbery': 3636}, Period('2011-06', 'M'): {'A

In [46]:
data = [{'Period': str(period), **crimes} for period, crimes in result_dict_crime.items()]
df = pd.DataFrame(data)

# Display the DataFrame
periodCrimeDataFrameDf = df.iloc[52:124]
periodCrimeDataFrameDf.isnull().any(axis=0)
# 72 is the number of rows
if  periodCrimeDataFrameDf["Public disorder and weapons"].isnull().sum() == 72 and periodCrimeDataFrameDf["Violent crime"].isnull().sum():
    print(True)


def ToDateTime(row):
    date_str = row["Period"]
    return pd.to_datetime(date_str,  format='%Y-%m')


periodCrimeDataFrameDf["Period"] = periodCrimeDataFrameDf.apply(ToDateTime, axis=1)


periodCrimeDataFrameDf['Period'] = periodCrimeDataFrameDf['Period'].dt.to_period('M')

#  if there are only missing values in thes comlumns they can be dropped

True


C:\Users\Marceli Morawski\AppData\Local\Temp\ipykernel_55252\1697383714.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  periodCrimeDataFrameDf["Period"] = periodCrimeDataFrameDf.apply(ToDateTime, axis=1)
C:\Users\Marceli Morawski\AppData\Local\Temp\ipykernel_55252\1697383714.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  periodCrimeDataFrameDf['Period'] = periodCrimeDataFrameDf['Period'].dt.to_period('M')


In [47]:
# dropping crime types that in the selceted time period had no values
periodCrimeDataFrameDf = periodCrimeDataFrameDf.drop(["Public disorder and weapons", "Violent crime"], axis=1) 


In [48]:
periodCrimeDataFrameDf

,Period,Anti-social behaviour,Other crime,Burglary,Vehicle crime,Robbery,Other theft,Criminal damage and arson,Drugs,Shoplifting,Violence and sexual offences,Theft from the person,Public order,Bicycle theft,Possession of weapons
52,2015-04,18327,685,5375,6569,1766,8457.0,5078.0,2591.0,3679.0,13536.0,3013.0,3302.0,1407.0,283.0
53,2015-05,19630,706,5524,6993,1882,9245.0,5348.0,2658.0,3627.0,15058.0,3200.0,3504.0,1663.0,285.0
54,2015-06,21392,734,5245,7261,1707,9383.0,5260.0,2780.0,3515.0,16562.0,2686.0,3720.0,1696.0,318.0
55,2015-07,23311,757,5368,6630,1764,9821.0,5405.0,2922.0,3430.0,17432.0,2650.0,3793.0,1762.0,316.0
56,2015-08,21826,683,5343,6454,1771,8927.0,4935.0,2961.0,3422.0,16176.0,2366.0,3561.0,1475.0,329.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2020-11,33833,964,5186,9096,2267,5663.0,4183.0,4360.0,2757.0,18697.0,2529.0,4200.0,1976.0,506.0
120,2020-12,29358,768,4740,7686,2002,6247.0,4062.0,3572.0,2679.0,17563.0,3031.0,3848.0,1303.0,383.0
121,2021-01,33611,727,4261,7629,1436,4776.0,3613.0,3644.0,2086.0,16048.0,1595.0,3418.0,1048.0,345.0
122,2021-02,29143,732,4035,7568,1311,4557.0,3537.0,4000.0,2229.0,15927.0,1493.0,3338.0,1102.0,375.0


In [49]:
periodCrimeDataFrameDf = periodCrimeDataFrameDf.drop(periodCrimeDataFrameDf[(periodCrimeDataFrameDf["Period"] == pd.Period('2017-02', freq='M')) | (periodCrimeDataFrameDf["Period"] == pd.Period('2017-03', freq='M'))].index)
 

In [50]:
periodCrimeDataFrameDf["Date"] = periodCrimeDataFrameDf["Period"]
periodCrimeDataFrameDf.drop(["Period"], axis=1, inplace=True)

# Merging the tables

In [142]:
model_df = pd.merge(dataPasModel, periodCrimeDataFrameDf, on='Date') 

In [151]:
cols = list(model_df.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('Proportion')) #Remove b from list
model_df = model_df[cols+['Proportion']]
model_df 
model_df.to_csv('merged_PAS_Crime.csv', index=False)

# Files for Confidence

## PAS for confidence Q61

In [51]:
selectecColumns = ["datetime", "Q61"]

Q61 = [["Very poor", "Poor", "Fair", "Good", "Excellent"]]


data_15_17_conf = data_15_17[selectecColumns]
data_17_18_conf = data_17_18[selectecColumns]
data_18_19_conf = data_18_19[selectecColumns]
data_19_20_conf = data_19_20[selectecColumns]
data_20_21_conf = data_20_21[selectecColumns]

frames = [data_15_17_conf, data_17_18_conf, data_18_19_conf, data_19_20_conf, data_20_21_conf] 

confData = pd.concat(frames)

# quick conversion to numbers for trust
ordinal_encoder(confData, Q61, "Q61", -1)

confData['datetime'] = confData['datetime'].dt.to_period('M')

confData

,datetime,Q61
0,2015-04,2.0
1,2015-04,3.0
2,2015-04,3.0
3,2015-04,3.0
4,2015-05,3.0
...,...,...
12731,2021-03,2.0
12732,2021-03,3.0
12733,2021-03,3.0
12734,2021-03,4.0


In [52]:
grouped_conf = confData.groupby(["datetime"])["Q61"].value_counts()

# getting the counts for the labels
result_dict_conf = {}
for (date, value), count in grouped_conf.items():
    if date not in result_dict_conf:
        result_dict_conf[date] = {}
    if value == -1: 
        pass
    else:
        result_dict_conf[date][value] = count

print(result_dict_conf)

{Period('2015-04', 'M'): {3.0: 586, 2.0: 272, 4.0: 87, 1.0: 50, 0.0: 10}, Period('2015-05', 'M'): {3.0: 642, 2.0: 272, 4.0: 73, 1.0: 28, 0.0: 11}, Period('2015-06', 'M'): {3.0: 619, 2.0: 301, 4.0: 86, 1.0: 47, 0.0: 13}, Period('2015-07', 'M'): {3.0: 599, 2.0: 286, 4.0: 92, 1.0: 41, 0.0: 13}, Period('2015-08', 'M'): {3.0: 601, 2.0: 262, 4.0: 62, 1.0: 44, 0.0: 8}, Period('2015-09', 'M'): {3.0: 622, 2.0: 293, 4.0: 82, 1.0: 60, 0.0: 7}, Period('2015-10', 'M'): {3.0: 561, 2.0: 266, 4.0: 81, 1.0: 58, 0.0: 17}, Period('2015-11', 'M'): {3.0: 603, 2.0: 305, 4.0: 90, 1.0: 42, 0.0: 13}, Period('2015-12', 'M'): {3.0: 710, 2.0: 222, 4.0: 110, 1.0: 35, 0.0: 7}, Period('2016-01', 'M'): {3.0: 681, 2.0: 223, 4.0: 84, 1.0: 38, 0.0: 7}, Period('2016-02', 'M'): {3.0: 1313, 2.0: 491, 4.0: 210, 1.0: 67, 0.0: 20}, Period('2016-03', 'M'): {3.0: 1303, 2.0: 467, 4.0: 200, 1.0: 65, 0.0: 16}, Period('2016-04', 'M'): {3.0: 680, 2.0: 223, 4.0: 94, 1.0: 28, 0.0: 6}, Period('2016-05', 'M'): {3.0: 665, 2.0: 257, 4.0: 

In [53]:
dataPasModelConf = PASPropportionDataFrame2(result_dict_conf)

In [54]:
dataPasModelConf

,Date,Proportion
0,2015-04,0.68
1,2015-05,0.70
2,2015-06,0.67
3,2015-07,0.68
4,2015-08,0.68
...,...,...
65,2020-11,0.51
66,2020-12,0.48
67,2021-01,0.48
68,2021-02,0.51


In [55]:
model_conf_df = pd.merge(dataPasModelConf, periodCrimeDataFrameDf, on='Date') 

In [56]:
cols = list(model_conf_df.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('Proportion')) #Remove b from list
model_conf_df = model_conf_df[cols+['Proportion']]
model_conf_df

,Date,Anti-social behaviour,Other crime,Burglary,Vehicle crime,Robbery,Other theft,Criminal damage and arson,Drugs,Shoplifting,Violence and sexual offences,Theft from the person,Public order,Bicycle theft,Possession of weapons,Proportion
0,2015-04,18327,685,5375,6569,1766,8457.0,5078.0,2591.0,3679.0,13536.0,3013.0,3302.0,1407.0,283.0,0.68
1,2015-05,19630,706,5524,6993,1882,9245.0,5348.0,2658.0,3627.0,15058.0,3200.0,3504.0,1663.0,285.0,0.70
2,2015-06,21392,734,5245,7261,1707,9383.0,5260.0,2780.0,3515.0,16562.0,2686.0,3720.0,1696.0,318.0,0.67
3,2015-07,23311,757,5368,6630,1764,9821.0,5405.0,2922.0,3430.0,17432.0,2650.0,3793.0,1762.0,316.0,0.68
4,2015-08,21826,683,5343,6454,1771,8927.0,4935.0,2961.0,3422.0,16176.0,2366.0,3561.0,1475.0,329.0,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2020-11,33833,964,5186,9096,2267,5663.0,4183.0,4360.0,2757.0,18697.0,2529.0,4200.0,1976.0,506.0,0.51
66,2020-12,29358,768,4740,7686,2002,6247.0,4062.0,3572.0,2679.0,17563.0,3031.0,3848.0,1303.0,383.0,0.48
67,2021-01,33611,727,4261,7629,1436,4776.0,3613.0,3644.0,2086.0,16048.0,1595.0,3418.0,1048.0,345.0,0.48
68,2021-02,29143,732,4035,7568,1311,4557.0,3537.0,4000.0,2229.0,15927.0,1493.0,3338.0,1102.0,375.0,0.51


In [58]:
model_conf_df.to_csv('./data/merged_PAS_Crine_conf.csv', index=False)